In [1]:
%run get_df_retractionwatch.ipynb

data/analysis/df_retracted_retraction_0_100.tsv
68
data/analysis/df_retracted_retraction_100_200.tsv
93
data/analysis/df_retracted_retraction_200_300.tsv
84
data/analysis/df_retracted_retraction_300_400.tsv
127
data/analysis/df_retracted_retraction_400_450.tsv
3


In [ ]:
df_articles_citing_retracted_paper.head(4)

,DOI_cited,article_citing
0,10.1136/ip.2006.015115,"<html>\n <body>\n <pub-id pub-id-type=""pmid"">..."
1,10.1136/ip.2006.015115,"<html>\n <body>\n <pub-id pub-id-type=""pmid"">..."
2,10.1186/s40478-016-0393-5,"<html>\n <body>\n <article article-type=""rese..."
3,10.1016/j.freeradbiomed.2011.11.027,<html>\n <body>\n <comment>\n S0891-\n </c...


# FIND PAPERS BEFORE AND AFTER RETRACTION, THE RETRACTION NOTE AND THE RETRACTION ITSELF. 
# YOU MAY CONSIDER TO  INTRODUCE A GAP BERORE AND AFTER RETRACTION

In [ ]:
from modules import Article_object

list_Articles_citing = []
list_Article_citing_dois = []
list_Article_citing_pmids = []
list_Article_citing_dates = []
list_Article_citing_references = []


for index, row in df_articles_citing_retracted_paper.iterrows():
    print(index)
    
    #for text in row['article_citing']:
    
    Article_citing = Article_object.Article_object()
    Article_citing.text = row['article_citing']

    Article_citing.get_doi_from_text()
    list_Article_citing_dois.append(Article_citing.doi)

    Article_citing.get_pmid_from_text()
    list_Article_citing_pmids.append(Article_citing.pmid)
    
    Article_citing.get_date_from_text()
    list_Article_citing_dates.append(Article_citing.date)

    Article_citing.get_list_references_dois_and_pmids_from_text()
    list_Article_citing_references.append(Article_citing.references_dois)
    
    

    list_Articles_citing.append(Article_citing)


0
1
2
PMID not found
3
DOI not found
Date not found
4
PMID not found
5
DOI not found
Date not found
6
7
8
DOI not found
9
PMID not found
10
PMID not found
11
12
PMID not found
13
Day or month out of range!!!
day:  
         31
         month:  
         11
         year:  
      2010
     
Date set to: 
day:  29 month:  
         11
         year:  
      2010
     
14
PMID not found
15
DOI not found
Date not found
16
17
DOI not found
Date not found
18
PMID not found
19
DOI not found
Date not found
20
21
PMID not found
22
23
DOI not found
Date not found
24
PMID not found
25
DOI not found
Date not found
26
27
28
29
DOI not found
Date not found
30
DOI not found
Date not found
31
32
PMID not found
33
DOI not found
Date not found
34
DOI not found
Date not found
35
DOI not found
Date not found
36
DOI not found
Date not found
37
DOI not found
Date not found
38
DOI not found
Date not found
39
DOI not found
Date not found
40
DOI not found
Date not found
41
DOI not found
Date not found
42
DOI n

In [ ]:
df_Articles_citing_retracted_paper = pd.concat([df_articles_citing_retracted_paper.reset_index(drop=True), 
                                    pd.DataFrame({"Article": list_Articles_citing,
                                            "date_article_citing": list_Article_citing_dates,
                                            "doi_article_citing": list_Article_citing_dois,
                                            "pmid_article_citing": list_Article_citing_pmids,
                                            "pmid_article_citing": list_Article_citing_pmids,
                                            "references_in_article_citing": list_Article_citing_references                     
                                            })
                                    ], axis=1)
df_Articles_citing_retracted_paper

# Let's add the date of the retraction
list_date_retracion = []
for  doi in df_Articles_citing_retracted_paper["DOI_cited"]:
    mask = df_retracted_retraction["doi"] == doi
    list_date_retracion.append(datetime.date(df_retracted_retraction[mask]["date_retraction"].iloc[0]))

df_Articles_citing =  pd.concat([df_Articles_citing_retracted_paper,
                                 pd.DataFrame({"date_doi_cited_retraction": list_date_retracion
                                               })
                                ], axis = 1)

In [ ]:
df_Articles_citing.shape

In [ ]:
df_Articles_citing.head(5)

In [ ]:
import numpy as np
df_Articles_citing.replace('', np.nan, inplace=True)
df_Articles_citing2 = df_Articles_citing.dropna(subset = ["date_article_citing"])
df_Articles_citing_clean = df_Articles_citing2.reset_index(drop = True)
df_Articles_citing_clean.head(5)

In [ ]:
df_Articles_citing_clean.shape

In [ ]:
list_index_is_the_retracted_paper =[]
list_index_is_the_retraction_note = []
for index, row in df_Articles_citing_clean.iterrows():   
    if row.doi_article_citing in list(df_retracted_retraction["doi"]):
        list_index_is_the_retracted_paper.append(index)
        
    if row.doi_article_citing in list(df_retracted_retraction["doi_retraction"]):
        list_index_is_the_retraction_note.append(index)
        #print(row.DOI_cited)
        #print(row.doi_article_citing)
        #print(index)

df_Articles_citing_no_retractions =  df_Articles_citing_clean.drop(list_index_is_the_retracted_paper + 
                                                                   list_index_is_the_retraction_note)


In [ ]:
df_Articles_citing_no_retractions.shape

In [ ]:
list_before_retraction = []
list_after_retraction = []

for index, row in df_Articles_citing_no_retractions.iterrows():
    if row['date_article_citing'] < row['date_doi_cited_retraction']:
        list_before_retraction.append(1)
        list_after_retraction.append(0)
    else:
        list_before_retraction.append(0)
        list_after_retraction.append(1)


df_Articles_before_after_retraction = pd.concat([df_Articles_citing_no_retractions.reset_index(drop = True),
                                                pd.DataFrame({"before_retraction": list_before_retraction,
                                                              "after_retraction": list_after_retraction
                                                             })
                                                ], axis = 1)
        

df_Articles_before_after_retraction.head(5)        


In [54]:
len(df_Articles_before_after_retraction["DOI_cited"].unique())

53

# Summary

61 different DOIs of retracted papers cited by
218 articles.

42 retracted papers, 25 retraction notes, and 152 new manuscripts.

152 new manuscripts:
     45 were published after the retraction
    107 before the retraction.

# Let's analyse the text and find the sentence were the paper was cited

In [ ]:
df_Articles_before_after_retraction.tail(1)

In [ ]:
df_Articles_before_after_retraction.head(3)

In [ ]:
import imp
import sys
sys.path.insert(0, "./modules")
from modules import get_df_analise_section
#imp.reload()

list_DOI_cited = df_Articles_before_after_retraction.DOI_cited.unique()

df_1sentence = pd.DataFrame() # To get only the sentence citing
df_3sentences = pd.DataFrame() # To get the sentence citing plus the following 3.

df_5sentences = pd.DataFrame()

for index, row in df_Articles_before_after_retraction.iterrows():
    
    df_analysis_row = get_df_analise_section.get_df(row.DOI_cited, [row.article_citing])
    #df_1sentence = df.append(df_analysis_row)
    df_1sentence = df_1sentence.append(df_analysis_row)

    
    df_3_analysis_row = get_df_analise_section.get_df(row.DOI_cited, [row.article_citing], 3) # We will get 3 additinal sentences
    df_3sentences = df_3sentences.append(df_3_analysis_row)
    
    df_5_analysis_row = get_df_analise_section.get_df(row.DOI_cited, [row.article_citing], 5) # We will get 3 additinal sentences
    df_5sentences = df_5sentences.append(df_5_analysis_row)
    
    
df_1sentence.reset_index(drop = True)
df_3sentences.reset_index(drop = True)

df_5sentences.reset_index(drop = True)

In [ ]:
df_1sentence.head(5)

In [ ]:
df_1sentence.columns

In [ ]:
len(df_1sentence.cited_DOI.unique())

In [ ]:
len(df_1sentence)

df.sentence_citing_discussion.iloc[0]

# Get Dataframe with preprocessed sentences

In [ ]:
#for index, row in df.iterrows():
import re
import numpy as np

"""
df_1sentence[['sentence_citing_conclusions',
       'sentence_citing_discussion', 'sentence_citing_intro',
       'sentence_citing_maintext']]
"""

def remove_citations(text):
    number_citations = text.count("[")
    while number_citations > 0:
        init = text.find("[") 
        end = text.find("]") 
        text = text.replace(text[init : end +1 ], " ") # replace the string between two delimiters
        number_citations -= 1
    return text

def remove_html_tags(text):
    to_clean = re.compile('<.*?>')
    clean_text = re.sub(to_clean, '', text)
    return clean_text
    

def get_df_sentences_preprocessed_and_classified(dataframe):
    
    list_all_citing_citing_sentences = []
    
    for index, row in dataframe.iterrows():

        list_sentences_citing_conclusions = row['sentence_citing_conclusions']
        list_sentences_citing_discussion = row['sentence_citing_discussion']
        list_sentences_citing_intro = row['sentence_citing_intro']
        list_sentences_citing_maintext = row['sentence_citing_maintext']

        if list_sentences_citing_conclusions != None:
            sentences_conclusions = " ".join(list_sentences_citing_conclusions)
            sentences_conclusions = remove_html_tags(sentences_conclusions)
            sentences_conclusions = remove_citations(sentences_conclusions)
        else:
            sentences_conclusions = ""

        if list_sentences_citing_discussion != None:
            sentences_discussion = " ".join(list_sentences_citing_discussion)
            sentences_discussion = remove_html_tags(sentences_discussion)
            sentences_discussion = remove_citations(sentences_discussion)

        else:
            sentences_discussion = ""

        if list_sentences_citing_intro != None:
            sentences_intro = " ".join(list_sentences_citing_intro)
            sentences_intro = remove_html_tags(sentences_intro)
            sentences_intro = remove_citations(sentences_intro)

        else:
            sentences_intro = ""

        if list_sentences_citing_maintext != None:
            sentences_maintext = " ".join(list_sentences_citing_maintext)
            sentences_maintext = remove_html_tags(sentences_maintext)
            sentences_maintext = remove_citations(sentences_maintext)
        else:
            sentences_maintext = ""

        all_sentences = (sentences_maintext + " " + 
                         sentences_discussion + " " +
                         sentences_intro + " " +
                         sentences_maintext)
        if all_sentences != "   ":
            list_all_citing_citing_sentences.append(all_sentences)
        else:
            list_all_citing_citing_sentences.append(np.nan)

    #print(all_sentences)

    dataframe_sentences_classification = pd.concat([df_Articles_before_after_retraction[["DOI_cited", 
                                                                                         "doi_article_citing",
                                                                                         #"pmid_article_citing",
                                                                                         "before_retraction",
                                                                                         "after_retraction"]],
                                            pd.DataFrame({"sentences_citing" : list_all_citing_citing_sentences
                                                          })
                                            ], axis = 1)
    return dataframe_sentences_classification

df1sentence_sentences_classification = get_df_sentences_preprocessed_and_classified(df_1sentence)
df3sentences_sentences_classification = get_df_sentences_preprocessed_and_classified(df_3sentences)

df5sentences_sentences_classification = get_df_sentences_preprocessed_and_classified(df_5sentences)



In [ ]:
##############TEST####################

In [ ]:


#for index, row in df.iterrows():
import re
import numpy as np

"""
df_1sentence[['sentence_citing_conclusions',
       'sentence_citing_discussion', 'sentence_citing_intro',
       'sentence_citing_maintext']]
"""

def remove_citations(text):
    number_citations = text.count("[")
    while number_citations > 0:
        init = text.find("[") 
        end = text.find("]") 
        text = text.replace(text[init : end +1 ], " ") # replace the string between two delimiters
        number_citations -= 1
    return text

def remove_html_tags(text):
    to_clean = re.compile('<.*?>')
    clean_text = re.sub(to_clean, '', text)
    return clean_text
    

def get_df_sentences_preprocessed_and_classified(dataframe):
    
    list_all_citing_citing_sentences = []
    
    for index, row in dataframe.iterrows():

        list_sentences_citing_conclusions = row['sentence_citing_conclusions']
        list_sentences_citing_discussion = row['sentence_citing_discussion']
        list_sentences_citing_intro = row['sentence_citing_intro']
        list_sentences_citing_maintext = row['sentence_citing_maintext']

        if list_sentences_citing_conclusions != None:
            sentences_conclusions = " ".join(list_sentences_citing_conclusions)
            sentences_conclusions = remove_html_tags(sentences_conclusions)
            #sentences_conclusions = remove_citations(sentences_conclusions)
        else:
            sentences_conclusions = ""

        if list_sentences_citing_discussion != None:
            sentences_discussion = " ".join(list_sentences_citing_discussion)
            sentences_discussion = remove_html_tags(sentences_discussion)
            #sentences_discussion = remove_citations(sentences_discussion)

        else:
            sentences_discussion = ""

        if list_sentences_citing_intro != None:
            sentences_intro = " ".join(list_sentences_citing_intro)
            sentences_intro = remove_html_tags(sentences_intro)
            #sentences_intro = remove_citations(sentences_intro)

        else:
            sentences_intro = ""

        if list_sentences_citing_maintext != None:
            sentences_maintext = " ".join(list_sentences_citing_maintext)
            sentences_maintext = remove_html_tags(sentences_maintext)
            #sentences_maintext = remove_citations(sentences_maintext)
        else:
            sentences_maintext = ""

        all_sentences = (sentences_maintext + " " + 
                         sentences_discussion + " " +
                         sentences_intro + " " +
                         sentences_maintext)
        if all_sentences != "   ":
            list_all_citing_citing_sentences.append(all_sentences)
        else:
            list_all_citing_citing_sentences.append(np.nan)

    #print(all_sentences)

    dataframe_sentences_classification = pd.concat([df_Articles_before_after_retraction[["DOI_cited", 
                                                                                         "doi_article_citing",
                                                                                         #"pmid_article_citing",
                                                                                         "before_retraction",
                                                                                         "after_retraction"]],
                                            pd.DataFrame({"sentences_citing" : list_all_citing_citing_sentences
                                                          })
                                            ], axis = 1)
    return dataframe_sentences_classification

df1sentence_sentences_classification = get_df_sentences_preprocessed_and_classified(df_1sentence)
df3sentences_sentences_classification = get_df_sentences_preprocessed_and_classified(df_3sentences)

df5sentences_sentences_classification = get_df_sentences_preprocessed_and_classified(df_5sentences)





In [ ]:
for sentence in df1sentence_sentences_classification.sentences_citing:
    print(sentence)

In [ ]:

###############  FIN TEST  #######################

In [44]:
len(df1sentence_sentences_classification)

122

In [39]:
df_Articles_before_after_retraction.head(1)

,DOI_cited,article_citing,Article,date_article_citing,doi_article_citing,pmid_article_citing,references_in_article_citing,date_doi_cited_retraction,after_retraction,before_retraction
0,10.1186/s13033-017-0143-5,<html>\n <body>\n <p>\n Int_J_Ment_Health_S...,<modules.Article_object.Article_object object ...,2017-11-30,NaN,28503193,NaN,2017-09-15,1,0


In [ ]:
df1sentence_sentences_classification.tail(3)

In [ ]:
#df.dropna(subset = "sentences_citing")

df1sentence_sentences_classification  = df1sentence_sentences_classification.dropna(subset = ["sentences_citing"])

df3sentences_sentences_classification  = df3sentences_sentences_classification.dropna(subset = ["sentences_citing"])


df5sentences_sentences_classification  = df5sentences_sentences_classification.dropna(subset = ["sentences_citing"])

In [ ]:
len(df1sentence_sentences_classification.doi_article_citing.unique())

In [ ]:
df1sentence_sentences_classification.head(3)


In [ ]:
len(df1sentence_sentences_classification.DOI_cited.unique())

# Let's split the dataframe on sentences before and after retraction

In [ ]:
df_sentences_before = df1sentence_sentences_classification[df1sentence_sentences_classification["before_retraction"] == 1]
df_sentences_after = df1sentence_sentences_classification[df1sentence_sentences_classification["after_retraction"] == 1]
print("df_sentences_before: ", df_sentences_before.shape)
print("df_sentences_after: ", df_sentences_after.shape)

df_3sentences_before = df3sentences_sentences_classification[df3sentences_sentences_classification["before_retraction"] == 1]
df_3sentences_after = df3sentences_sentences_classification[df3sentences_sentences_classification["after_retraction"] == 1]

df_5sentences_before = df5sentences_sentences_classification[df5sentences_sentences_classification["before_retraction"] == 1]
df_5sentences_after = df5sentences_sentences_classification[df5sentences_sentences_classification["after_retraction"] == 1]


In [45]:
len(df_sentences_after.DOI_cited.unique())

24

In [ ]:
for sentence in df_sentences_after.sentences_citing:
    print(sentence)

# Let's analyse the documents

* How many the documents from df_sentences_before cite DOIs that are not cited after retraction? 
* How many cite DOIs that are also cited after retraction?

* How many the documents from df_sentences_after cite DOIs that are not cited before retraction? 
* How many cite DOIs that are also cited before retraction?


In [ ]:
dois_cited_before = set(df_sentences_before["DOI_cited"])
dois_cited_after = set(df_sentences_after["DOI_cited"])

doi_cited_before_and_after = dois_cited_before.intersection(dois_cited_after)
doi_cited_before_and_after 

doi_cited_only_before = dois_cited_before.difference(dois_cited_after)
doi_cited_only_before

doi_cited_only_after = dois_cited_after.difference(dois_cited_before)
doi_cited_only_after

print("\n unique dois_cited_before ", len(dois_cited_before))
print("\n unique dois_cited_after ", len(dois_cited_after))
print("\n unique doi_cited_before_and_after ", len(doi_cited_before_and_after))
print("\n unqiue doi_cited_only_before ", len(doi_cited_only_before))
print("\n unique doi_cited_only_after ", len(doi_cited_only_after))



In [ ]:
print("\n number documents citing after - DOIS that are only cited after: ")
df_sentences_after_only = df_sentences_after[df_sentences_after.DOI_cited.isin(doi_cited_only_after)]
print(df_sentences_after_only.shape[0])

print("\n number documents citing after - DOIS that are cited before and after:")
df_sentences_after_not_only = df_sentences_after[df_sentences_after.DOI_cited.isin(doi_cited_before_and_after)]
print(df_sentences_after_not_only.shape[0])

print("\n number documents citing before - DOIS that are only cited before: ")
df_sentences_before_only = df_sentences_before[df_sentences_before.DOI_cited.isin(doi_cited_only_before)]
print(df_sentences_before_only.shape[0])

print("\n number documents citing before - DOIS that are cited before and after:")
df_sentences_before_not_only = df_sentences_before[df_sentences_before.DOI_cited.isin(doi_cited_before_and_after)]
print(df_sentences_before_not_only.shape[0])

########################3

df_3sentences_after_only = df_3sentences_after[df_3sentences_after.DOI_cited.isin(doi_cited_only_after)]

df_3sentences_after_not_only = df_3sentences_after[df_3sentences_after.DOI_cited.isin(doi_cited_before_and_after)]

df_3sentences_before_only = df_3sentences_before[df_3sentences_before.DOI_cited.isin(doi_cited_only_before)]

df_3sentences_before_not_only = df_3sentences_before[df_3sentences_before.DOI_cited.isin(doi_cited_before_and_after)]


#######################

df_5sentences_after_only = df_5sentences_after[df_5sentences_after.DOI_cited.isin(doi_cited_only_after)]

df_5sentences_after_not_only = df_5sentences_after[df_5sentences_after.DOI_cited.isin(doi_cited_before_and_after)]

df_5sentences_before_only = df_5sentences_before[df_5sentences_before.DOI_cited.isin(doi_cited_only_before)]

df_5sentences_before_not_only = df_5sentences_before[df_5sentences_before.DOI_cited.isin(doi_cited_before_and_after)]


In [40]:
df_sentences_before.shape

(90, 5)

### Summary 3:

90 documents before retractions:
    
    - 53 documents cite DOIs (23 unique DOIs) that are not cited after retraction.
    - 37 documents cite DOIs (12 unique DOIs) that are also cited after retraction.

    - 83 unique doi_article_citing - len(df_sentences_before.doi_article_citing.unique())
        There are 7 documents without the DOIS_citing     
    - 35 unique DOI_cited - len(df_sentences_before.DOI_cited.unique())
        Of those 23 DOI are only cited before. 12 before and after.


32 documents after retraction:
    
    - 15 documents cite DOIs (12 unique DOIs) that are not cited before retraction.
    - 17 documents cite DOIs (12 unique DOIs) that are also cited before retraction.


    - 29 unique doi_article_citing - len(df_sentences_after.doi_article_citing.unique())
        There are 3 documents without the DOIS_citing        
    - 24 unique DOI_cited - len(df_sentences_after.DOI_cited.unique())
        Of those 12 DOI are only cited after. 12 before and after.
    

We will select documents before retraction only if they also have been cited after retraction.


#####################################################################################

# Summary 2 

61 different DOIs of retracted papers cited by
218 articles.

42 retracted papers, 25 retraction notes, and 152 new manuscripts.

152 new manuscripts:
     45 were published after the retraction
    107 before the retraction.
    
* The algorithm is able to find the text citing in:
    
127 manuscripts:
    36 were published after retraction
    91 were published before retraction


# Selection of documents, stemming and counting of words to train a classifier

In [95]:
import imp
#imp.reload('modules.get_stemmed_sentences_and_list_words')
#import importlib

#import modules.get_stemmed_sentences_and_list_words
#from modules.get_stemmed_sentences_and_list_words import get_tuple_articles_stemmed_classified_and_list_words # get_stemmed_sentences_and_list_words
#import imp
#from modules.get_stemmed_sentences_and_list_words import get_tuple_articles_stemmed_classified_and_list_words # get_stemmed_sentences_and_list_words

#path = "/project/eLife_retractions/modules"
#os.chdir(path)
#importlib.reload("get_stemmed_sentences_and_list_words.py")

from modules.get_stemmed_sentences_and_list_words import *
#from modules.get_stemmed_sentences_and_list_words import get_tuple_articles_stemmed_classified_and_list_words # get_stemmed_sentences_and_list_words
import imp
imp.reload(modules.get_stemmed_sentences_and_list_words)



# 1. With one sentence citing:

# 1.1 -  All documents
sentences_before = df_sentences_before.sentences_citing
sentences_after = df_sentences_after.sentences_citing
(documents_all, list_all_words, list_all_bigrams) = get_tuple_articles_stemmed_classified_and_list_words(sentences_before, sentences_after)


# 1.2 -  50 % before - 50% after
#documents = document_before[:len(document_after)] + document_after
sentences_before = df_sentences_before.sentences_citing
sentences_after = df_sentences_after.sentences_citing
sentences_before_part = sentences_before[:len(sentences_after)]
(documents_50_50, list_all_words_50_50, list_all_bigrams_50_50) = get_tuple_articles_stemmed_classified_and_list_words(sentences_before_part, sentences_after)


# 2. With one sentence citing + following 3 sentences:

# 2.1 -  All documents
#documents_3_all = document_3_before + document_3_after
sentences_before = df_3sentences_before.sentences_citing
sentences_after = df_3sentences_after.sentences_citing
(documents_all_3sentences, list_all_words_3sentences, list_all_bigrams_3sentences) = get_tuple_articles_stemmed_classified_and_list_words(sentences_before, sentences_after)


# 2.2 -  50 % before - 50% after
#documents = document_before[:len(document_after)] + document_after
sentences_before = df_3sentences_before.sentences_citing
sentences_after = df_3sentences_after.sentences_citing
sentences_before_part = sentences_before[:len(sentences_after)]
(documents_50_50_3sentences, list_all_words_50_50_3sentences, list_all_bigrams_50_50_3sentences) = get_tuple_articles_stemmed_classified_and_list_words(sentences_before_part, sentences_after)

  
    


In [98]:
list_all_bigrams_50_50_3sentences

[('abbrevi', 'mirp'),
 ('abbrevi', 'mirp'),
 ('abbrevi', 'mirp'),
 ('aberr', 'bcell'),
 ('aberr', 'bcell'),
 ('aberr', 'bcell'),
 ('abil', 'interact'),
 ('abil', 'interact'),
 ('abil', 'interact'),
 ('abil', 'mira'),
 ('abil', 'mira'),
 ('abil', 'mira'),
 ('abil', 'nor'),
 ('abil', 'nor'),
 ('abl', 'alter'),
 ('abl', 'alter'),
 ('abl', 'alter'),
 ('abl', 'assembl'),
 ('abl', 'assembl'),
 ('abl', 'assembl'),
 ('abl', 'boost'),
 ('abl', 'grow'),
 ('abl', 'grow'),
 ('abl', 'grow'),
 ('abl', 'induc'),
 ('abl', 'induc'),
 ('abl', 'induc'),
 ('abl', 'induc'),
 ('ablat', 'betatrophin'),
 ('ablat', 'betatrophin'),
 ('ablat', 'betatrophin'),
 ('abnorm', 'astrogenesi'),
 ('abnorm', 'cuticl'),
 ('abnorm', 'cuticl'),
 ('abnorm', 'cuticl'),
 ('abnorm', 'express'),
 ('abnorm', 'express'),
 ('abnorm', 'express'),
 ('abnorm', 'high'),
 ('abnorm', 'high'),
 ('abnorm', 'high'),
 ('absenc', 'lead'),
 ('absenc', 'lead'),
 ('abstract', 'introduct'),
 ('abstract', 'introduct'),
 ('abund', 'shown'),
 ('abund

In [99]:
# 3. With one sentence citing + following 5 sentences:

# 3.1 -  All documents
#documents_3_all = document_3_before + document_3_after
sentences_before = df_5sentences_before.sentences_citing
sentences_after = df_5sentences_after.sentences_citing
(documents_all_5sentences, list_all_words_5sentences, list_all_bigrams_5sentences) = get_tuple_articles_stemmed_classified_and_list_words(sentences_before, sentences_after)


# 3.2 -  50 % before - 50% after
#documents = document_before[:len(document_after)] + document_after
sentences_before = df_5sentences_before.sentences_citing
sentences_after = df_5sentences_after.sentences_citing
sentences_before_part = sentences_before[:len(sentences_after)]
(documents_50_50_5sentences, list_all_words_50_50_5sentences, list_all_bigrams_50_50_5sentences) = get_tuple_articles_stemmed_classified_and_list_words(sentences_before_part, sentences_after)


In [100]:
print(len(list_all_words),
len(list_all_words_50_50), 
len(list_all_words_3sentences),
len(list_all_words_50_50_3sentences))

10630 6007 32659 16912


In [ ]:
len(list_all_words_50_50_5sentences)

# Selection: same number of documents_citing before_only than after_only, and same number of documents citing the same DOI before and ater


In [101]:
# 1.3. Same number of documents_citing before_only than after_only, and same number of documents citing the same DOI before and ater

# Let's get all the papers from df_sentences_after_only (X), and from df_sentences_after_not_only (Y)
# and X papers from df_sentences_before_only, and Y documents from df_sentences_before_and_after. 
# When selecting the documents for df_sentences_before_and_after we must be sure that we are not selecting 
# all the papers the same doi (for each cited doi, we want to have the number of douments citing before and 
# after)

df_sentences_before_only_selection = df_sentences_before_only.sample(frac=1) #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
df_sentences_before_only_selection = df_sentences_before_only_selection.drop_duplicates(subset = "DOI_cited")
df_sentences_before_only_selection = df_sentences_before_only_selection[:df_sentences_after_only.shape[0]]
df_sentences_before_only_selection

list_doi_added = []
df_sentences_before_not_only_selection = pd.DataFrame()
df_sentences_after_not_only_selection = pd.DataFrame()
#more_dois_in_

for doi in list(df_sentences_after_not_only.DOI_cited.unique()):

    #times_doi_in_df_sentences_after_not_only = list(df_sentences_after_not_only.DOI_cited).count(doi)
    
    df_sentences_after_not_only_doi_selected = df_sentences_after_not_only[df_sentences_after_not_only["DOI_cited"] == doi]
    times_doi_in_df_sentences_after_not_only =  df_sentences_after_not_only_doi_selected.shape[0]
    
    df_sentences_before_not_only_doi_selected = df_sentences_before_not_only[df_sentences_before_not_only["DOI_cited"] == doi]
    times_doi_in_df_sentences_before_not_only = df_sentences_before_not_only_doi_selected.shape[0]

    """    print(doi)
    print("times_doi in df_sentences_after_not_only: ", times_doi_in_df_sentences_after_not_only)
    print("times_doi in df_sentences_before_not_only: ", times_doi_in_df_sentences_before_not_only)
    """    
    #threshold_difference_in_number = 0 # if you want to allow X documents more before/after or after/before
    if times_doi_in_df_sentences_after_not_only <= times_doi_in_df_sentences_before_not_only:
        df_sentences_before_not_only_selection = df_sentences_before_not_only_selection.append(
                                  df_sentences_before_not_only[:times_doi_in_df_sentences_after_not_only])
        df_sentences_after_not_only_selection = df_sentences_after_not_only_selection.append(
                                  df_sentences_after_not_only[:times_doi_in_df_sentences_after_not_only])
    else: 
        df_sentences_before_not_only_selection = df_sentences_before_not_only_selection.append(
                                  df_sentences_before_not_only[:times_doi_in_df_sentences_before_not_only])
        df_sentences_after_not_only_selection = df_sentences_after_not_only_selection.append(
                                  df_sentences_after_not_only[:times_doi_in_df_sentences_before_not_only])

# The dataframes with the selected documents are:
# df_sentences_before_not_only_selection
# df_sentences_after_not_only_selection
# df_sentences_before_only_selection
# df_sentences_after_only

df_sentences_before_selection = df_sentences_before_only_selection.append(df_sentences_before_not_only_selection)
df_sentences_after_selection = df_sentences_after_only.append(df_sentences_after_not_only_selection)

sentences_before = df_sentences_before_selection.sentences_citing
sentences_after = df_sentences_after_selection.sentences_citing
(documents_50_50_balanced, list_all_words_50_50_balanced, list_all_bigrams_50_50_balanced) = get_tuple_articles_stemmed_classified_and_list_words(sentences_before, sentences_after)


In [ ]:
df_sentences_before_not_only_selection.shape

In [ ]:
df_sentences_after_not_only_selection.shape

In [ ]:
df_sentences_before_selection.shape

In [ ]:
df_sentences_after_selection.shape

In [ ]:
len(documents_50_50_balanced)

In [102]:
# 3.3. Same number of documents_citing before_only than after_only, and same number of documents citing the same DOI before and ater

df_3sentences_before_only_selection = df_3sentences_before_only.sample(frac=1) #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
df_3sentences_before_only_selection = df_3sentences_before_only_selection.drop_duplicates(subset = "DOI_cited")
df_3sentences_before_only_selection = df_3sentences_before_only_selection[:df_3sentences_after_only.shape[0]]
df_3sentences_before_only_selection

list_doi_added = []
df_3sentences_before_not_only_selection = pd.DataFrame()
df_3sentences_after_not_only_selection = pd.DataFrame()
#more_dois_in_

for doi in list(df_3sentences_after_not_only.DOI_cited.unique()):

    #times_doi_in_df_sentences_after_not_only = list(df_sentences_after_not_only.DOI_cited).count(doi)
    
    df_3sentences_after_not_only_doi_selected = df_3sentences_after_not_only[df_3sentences_after_not_only["DOI_cited"] == doi]
    times_doi_in_df_3sentences_after_not_only =  df_3sentences_after_not_only_doi_selected.shape[0]
    
    df_3sentences_before_not_only_doi_selected = df_3sentences_before_not_only[df_3sentences_before_not_only["DOI_cited"] == doi]
    times_doi_in_df_3sentences_before_not_only = df_3sentences_before_not_only_doi_selected.shape[0]

    """    print(doi)
    print("times_doi in df_sentences_after_not_only: ", times_doi_in_df_sentences_after_not_only)
    print("times_doi in df_sentences_before_not_only: ", times_doi_in_df_sentences_before_not_only)
    """    
    #threshold_difference_in_number = 0 # if you want to allow X documents more before/after or after/before
    if times_doi_in_df_3sentences_after_not_only <= times_doi_in_df_3sentences_before_not_only:
        df_3sentences_before_not_only_selection = df_3sentences_before_not_only_selection.append(
                                  df_3sentences_before_not_only[:times_doi_in_df_3sentences_after_not_only])
        df_3sentences_after_not_only_selection = df_3sentences_after_not_only_selection.append(
                                  df_3sentences_after_not_only[:times_doi_in_df_3sentences_after_not_only])
    else: 
        df_3sentences_before_not_only_selection = df_3sentences_before_not_only_selection.append(
                                  df_3sentences_before_not_only[:times_doi_in_df_3sentences_before_not_only])
        df_3sentences_after_not_only_selection = df_3sentences_after_not_only_selection.append(
                                  df_3sentences_after_not_only[:times_doi_in_df_3sentences_before_not_only])

# The dataframes with the selected documents are:
# df_3sentences_before_not_only_selection
# df_3sentences_after_not_only_selection
# df_3sentences_before_only_selection
# df_3sentences_after_only

df_3sentences_before_balanced = df_3sentences_before_only_selection.append(df_3sentences_before_not_only_selection)
df_3sentences_after_balanced = df_3sentences_after_only.append(df_3sentences_after_not_only_selection)


sentences_before = df_3sentences_before_balanced.sentences_citing
sentences_after = df_3sentences_after_balanced.sentences_citing
(documents_50_50_balanced_3sentences, list_all_words_50_50_balanced_3sentences, list_all_bigrams_50_50_balanced_3sentences) = get_tuple_articles_stemmed_classified_and_list_words(sentences_before, sentences_after)


In [ ]:
df_3sentences_before_balanced.shape

In [ ]:
df_3sentences_after_balanced.shape

In [103]:
# 3.3. Same number of documents_citing before_only than after_only, and same number of documents citing the same DOI before and ater

df_5sentences_before_only_selection = df_5sentences_before_only.sample(frac=1) #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
df_5sentences_before_only_selection = df_5sentences_before_only_selection.drop_duplicates(subset = "DOI_cited")
df_5sentences_before_only_selection = df_5sentences_before_only_selection[:df_5sentences_after_only.shape[0]]
df_5sentences_before_only_selection

list_doi_added = []
df_5sentences_before_not_only_selection = pd.DataFrame()
df_5sentences_after_not_only_selection = pd.DataFrame()
#more_dois_in_

for doi in list(df_5sentences_after_not_only.DOI_cited.unique()):

    #times_doi_in_df_sentences_after_not_only = list(df_sentences_after_not_only.DOI_cited).count(doi)
    
    df_5sentences_after_not_only_doi_selected = df_5sentences_after_not_only[df_5sentences_after_not_only["DOI_cited"] == doi]
    times_doi_in_df_5sentences_after_not_only =  df_5sentences_after_not_only_doi_selected.shape[0]
    
    df_5sentences_before_not_only_doi_selected = df_5sentences_before_not_only[df_5sentences_before_not_only["DOI_cited"] == doi]
    times_doi_in_df_5sentences_before_not_only = df_5sentences_before_not_only_doi_selected.shape[0]

    """    print(doi)
    print("times_doi in df_sentences_after_not_only: ", times_doi_in_df_sentences_after_not_only)
    print("times_doi in df_sentences_before_not_only: ", times_doi_in_df_sentences_before_not_only)
    """    
    #threshold_difference_in_number = 0 # if you want to allow X documents more before/after or after/before
    if times_doi_in_df_5sentences_after_not_only <= times_doi_in_df_5sentences_before_not_only:
        df_5sentences_before_not_only_selection = df_5sentences_before_not_only_selection.append(
                                  df_5sentences_before_not_only[:times_doi_in_df_5sentences_after_not_only])
        df_5sentences_after_not_only_selection = df_5sentences_after_not_only_selection.append(
                                  df_5sentences_after_not_only[:times_doi_in_df_5sentences_after_not_only])
    else: 
        df_5sentences_before_not_only_selection = df_5sentences_before_not_only_selection.append(
                                  df_5sentences_before_not_only[:times_doi_in_df_5sentences_before_not_only])
        df_5sentences_after_not_only_selection = df_5sentences_after_not_only_selection.append(
                                  df_5sentences_after_not_only[:times_doi_in_df_5sentences_before_not_only])

# The dataframes with the selected documents are:
# df_3sentences_before_not_only_selection
# df_3sentences_after_not_only_selection
# df_3sentences_before_only_selection
# df_3sentences_after_only

df_5sentences_before_balanced = df_3sentences_before_only_selection.append(df_5sentences_before_not_only_selection)
df_5sentences_after_balanced = df_3sentences_after_only.append(df_5sentences_after_not_only_selection)


sentences_before = df_5sentences_before_balanced.sentences_citing
sentences_after = df_5sentences_after_balanced.sentences_citing
(documents_50_50_balanced_5sentences, list_all_words_50_50_balanced_5sentences, list_all_bigrams_50_50_balanced_5sentences) = get_tuple_articles_stemmed_classified_and_list_words(sentences_before, sentences_after)


In [105]:
len(list_all_bigrams)

10509

# SUM UP:


### 1. With one sentence citing:

#### 1.1 -  All documents
* documents_all
* list_all_words
* list_all_bigrams

#### 1.2 -  50 % before - 50% after
* documents_50_50
* list_all_words_50_50
* list_all_bigrams_50_50

#### 1.3 - Balanced: 50 % before - 50% after. If doi cited before and after same number of documents before are after.
* documents_50_50_balanced
* list_all_words_50_50_balanced
* list_all_bigrams_50_50_balanced 

### 2. With one sentence citing + following 3 sentences:

#### 2.1 -  All documents
* documents_all_3sentences
* list_all_words_3sentences
* list_all_bigrams_3sentences

#### 2.2 -  50 % before - 50% after
* documents_50_50_3sentences
* list_all_words_50_50_3sentences
* list_all_bigrams_50_50_3sentences

#### 2.3 - Balanced: 50 % before - 50% after. If doi cited before and after same number of documents before are after.
* documents_50_50_balanced_3sentences
* list_all_words_50_50_balanced_3sentences
* list_all_bigrams_50_50_balanced_3sentences
    

# Naive Bayes Algorithm

### Some checkings

In [ ]:
print("Number of words in : \n\nlist_all_words: \t", len(list_all_words),
    "\nlist_all_words_50_50: \t", len(list_all_words_50_50), 
    "\nlist_all_words_50_50_balanced: \t", len(list_all_words_50_50_balanced),
    "\n\nlist_all_words_3sentences: \t", len(list_all_words_3sentences),
    "\nlist_all_words_50_50_3sentences: \t", len(list_all_words_50_50_3sentences),
    "\nlist_all_words_50_50_balanced_3sentences: \t", len(list_all_words_50_50_balanced_3sentences)
     )

### Naive Bayes

In [137]:
import nltk 

def find_features(document, word_features):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

def train_Naive_Bayes(documents, list_all_words, spliting_training_test_ratio = 0.25):
    
    # Word frequencies
    all_words_freq = nltk.FreqDist(list_all_words)
    word_features = list(all_words_freq.keys()) # Those are all teh words in the document. Each word is a feature

    featuresets = [(find_features(article, word_features), cat) for (article, cat) in documents]

    # set that we'll train our classifier with                    
    test_size = round(spliting_training_test_ratio * len(featuresets))
    print("documents in test_set: \t", test_size)
    print("documents in training_set: \t", len(featuresets) -test_size)
    
    training_set = featuresets[test_size:]

    # set that we'll test against.
    testing_set = featuresets[:test_size]

    classifier = nltk.NaiveBayesClassifier.train(training_set)
    print("\nClassifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
    
    classifier.show_most_informative_features(100)
    
    

                         
                         
"""
list_all_words_50_50_freq = nltk.FreqDist(list_all_words_50_50) 
all_words_50_50_features = list(list_all_words_50_50_freq.keys())

list_all_words_50_50_balanced_freq = nltk.FreqDist(list_all_words_50_50_balanced)
all_words_50_50_balanced_features =  nltk.FreqDist(list_all_words_50_50_balanced)

list_all_words_3sentences_freq = nltk.FreqDist(list_all_words_3sentences)

list_all_words_50_50_3sentences_freq = nltk.FreqDist(list_all_words_50_50_3sentences)

list_all_words_50_50_balanced_3sentences_freq = nltk.FreqDist(list_all_words_50_50_balanced_3sentences)
"""

'\nlist_all_words_50_50_freq = nltk.FreqDist(list_all_words_50_50) \nall_words_50_50_features = list(list_all_words_50_50_freq.keys())\n\nlist_all_words_50_50_balanced_freq = nltk.FreqDist(list_all_words_50_50_balanced)\nall_words_50_50_balanced_features =  nltk.FreqDist(list_all_words_50_50_balanced)\n\nlist_all_words_3sentences_freq = nltk.FreqDist(list_all_words_3sentences)\n\nlist_all_words_50_50_3sentences_freq = nltk.FreqDist(list_all_words_50_50_3sentences)\n\nlist_all_words_50_50_balanced_3sentences_freq = nltk.FreqDist(list_all_words_50_50_balanced_3sentences)\n'

In [138]:
### 1. With one sentence citing:

#### 1.1 -  All documents

train_Naive_Bayes(documents_all, list_all_words, 0.20)

documents in test_set: 	 24
documents in training_set: 	 98

Classifier accuracy percent: 75.0
Most Informative Features
                   serum = True            after : before =      8.1 : 1.0
                  associ = True            after : before =      6.3 : 1.0
                    gene = True           before : after  =      4.8 : 1.0
                    need = True            after : before =      4.5 : 1.0
                 upregul = True            after : before =      4.5 : 1.0
                     msc = True            after : before =      4.5 : 1.0
                     due = True            after : before =      4.5 : 1.0
                  surviv = True            after : before =      4.5 : 1.0
                    show = True            after : before =      4.2 : 1.0
                 increas = True            after : before =      3.9 : 1.0
                    role = True           before : after  =      3.6 : 1.0
                     fli = True            after : bef

In [108]:
train_Naive_Bayes(documents_all, list_all_bigrams, 0.20)

documents in test_set: 	 24
documents in training_set: 	 98

Classifier accuracy percent: 75.0
Most Informative Features
('complex', 'minicellulosom') = False           after : before =      1.0 : 1.0
       ('the', 'reason') = False           after : before =      1.0 : 1.0
      ('implic', 'lupu') = False           after : before =      1.0 : 1.0
    ('polytrop', 'mulv') = False           after : before =      1.0 : 1.0
  ('cytokin', 'monocyt') = False           after : before =      1.0 : 1.0
('demonstr', 'compound') = False           after : before =      1.0 : 1.0
    ('organ', 'nanowir') = False           after : before =      1.0 : 1.0
('superscript', 'platinum') = False           after : before =      1.0 : 1.0
  ('regul', 'metastasi') = False           after : before =      1.0 : 1.0
    ('improv', 'glucos') = False           after : before =      1.0 : 1.0
  ('adipocyt', 'produc') = False           after : before =      1.0 : 1.0
('synthes', 'decarboxyl') = False           af

In [109]:
#### 1.2 -  50 % before - 50% after

train_Naive_Bayes(documents_50_50, list_all_words_50_50, 0.20)


documents in test_set: 	 13
documents in training_set: 	 51

Classifier accuracy percent: 69.23076923076923
Most Informative Features
                     use = True            after : before =      6.3 : 1.0
                   mirna = True           before : after  =      6.0 : 1.0
                    role = True           before : after  =      5.2 : 1.0
                     mir = True           before : after  =      4.4 : 1.0
                    gene = True           before : after  =      3.6 : 1.0
                  target = True           before : after  =      3.6 : 1.0
                prolifer = True           before : after  =      3.6 : 1.0
                   human = True           before : after  =      3.6 : 1.0
                  signal = True           before : after  =      3.1 : 1.0
                  factor = True           before : after  =      2.8 : 1.0
                 possibl = True           before : after  =      2.8 : 1.0
                    mous = True          

In [130]:
train_Naive_Bayes(documents_50_50, list_all_bigrams_50_50, 0.20)


documents in test_set: 	 13
documents in training_set: 	 51

Classifier accuracy percent: 30.76923076923077
Most Informative Features
('complex', 'minicellulosom') = False          before : after  =      1.0 : 1.0
       ('the', 'reason') = False          before : after  =      1.0 : 1.0
      ('implic', 'lupu') = False          before : after  =      1.0 : 1.0
    ('organ', 'nanowir') = False          before : after  =      1.0 : 1.0
  ('regul', 'metastasi') = False          before : after  =      1.0 : 1.0
    ('improv', 'glucos') = False          before : after  =      1.0 : 1.0
('synthes', 'decarboxyl') = False          before : after  =      1.0 : 1.0
('proinvas', 'prometastat') = False          before : after  =      1.0 : 1.0
   ('ila', 'phenomenon') = False          before : after  =      1.0 : 1.0
        ('area', 'boil') = False          before : after  =      1.0 : 1.0
   ('wrong', 'antibodi') = False          before : after  =      1.0 : 1.0
      ('offici', 'name') = False

In [111]:
#### 1.3 - Balanced: 50 % before - 50% after. If doi cited before and after same number of documents before are after.

train_Naive_Bayes(documents_50_50_balanced, list_all_words_50_50_balanced, 0.20)


documents in test_set: 	 12
documents in training_set: 	 50

Classifier accuracy percent: 83.33333333333334
Most Informative Features
                  enhanc = True            after : before =      7.7 : 1.0
             overexpress = True            after : before =      6.6 : 1.0
                  secret = True            after : before =      6.0 : 1.0
                   known = True            after : before =      6.0 : 1.0
                    gene = True           before : after  =      5.8 : 1.0
                  effect = True            after : before =      4.0 : 1.0
                 develop = True           before : after  =      3.5 : 1.0
                   liver = True            after : before =      3.3 : 1.0
                  glucos = True            after : before =      3.3 : 1.0
             betatrophin = True            after : before =      3.3 : 1.0
                    mice = True            after : before =      2.8 : 1.0
                   howev = True          

In [112]:
train_Naive_Bayes(documents_50_50_balanced, list_all_bigrams_50_50_balanced, 0.20)


documents in test_set: 	 12
documents in training_set: 	 50

Classifier accuracy percent: 33.33333333333333
Most Informative Features
      ('data', 'provid') = False           after : before =      1.0 : 1.0
       ('the', 'reason') = False           after : before =      1.0 : 1.0
('extracellular', 'αsubunit') = False           after : before =      1.0 : 1.0
('demonstr', 'compound') = False           after : before =      1.0 : 1.0
    ('organ', 'nanowir') = False           after : before =      1.0 : 1.0
('ubiquitinbind', 'data') = False           after : before =      1.0 : 1.0
     ('βcell', 'replic') = False           after : before =      1.0 : 1.0
     ('research', 'lab') = False           after : before =      1.0 : 1.0
        ('lab', 'focus') = False           after : before =      1.0 : 1.0
    ('improv', 'glucos') = False           after : before =      1.0 : 1.0
  ('pattern', 'similar') = False           after : before =      1.0 : 1.0
      ('state', 'mutat') = False   

In [113]:
### 2. With one sentence citing + following 3 sentences:

#### 2.1 -  All documents

train_Naive_Bayes(documents_all_3sentences, list_all_words_3sentences, 0.20)


documents in test_set: 	 24
documents in training_set: 	 98

Classifier accuracy percent: 75.0
Most Informative Features
                 univers = True            after : before =      6.0 : 1.0
                   debat = True            after : before =      6.0 : 1.0
                 mainten = True            after : before =      6.0 : 1.0
                     msc = True            after : before =      6.0 : 1.0
                   liver = True            after : before =      6.0 : 1.0
                     led = True            after : before =      6.0 : 1.0
                  despit = True            after : before =      4.8 : 1.0
                 retract = True            after : before =      4.6 : 1.0
                   valid = True            after : before =      4.6 : 1.0
                     via = True            after : before =      4.3 : 1.0
                   modif = True            after : before =      4.3 : 1.0
                  replac = True            after : bef

In [114]:

train_Naive_Bayes(documents_all_3sentences, list_all_bigrams_3sentences, 0.20)


documents in test_set: 	 24
documents in training_set: 	 98

Classifier accuracy percent: 79.16666666666666
Most Informative Features
('complex', 'minicellulosom') = False           after : before =      1.0 : 1.0
        ('estim', 'use') = False           after : before =      1.0 : 1.0
       ('not', 'involv') = False           after : before =      1.0 : 1.0
       ('the', 'reason') = False           after : before =      1.0 : 1.0
 ('whether', 'directli') = False           after : before =      1.0 : 1.0
        ('rate', 'fold') = False           after : before =      1.0 : 1.0
  ('relev', 'pulmonari') = False           after : before =      1.0 : 1.0
   ('splice', 'variant') = False           after : before =      1.0 : 1.0
       ('level', 'vari') = False           after : before =      1.0 : 1.0
        ('the', 'first') = False           after : before =      1.0 : 1.0
       ('regen', 'drug') = False           after : before =      1.0 : 1.0
       ('mechan', 'the') = False    

In [115]:
#### 2.2 -  50 % before - 50% after

train_Naive_Bayes(documents_50_50_3sentences, list_all_words_50_50_3sentences, 0.20)
    

documents in test_set: 	 13
documents in training_set: 	 51

Classifier accuracy percent: 69.23076923076923
Most Informative Features
                     mir = True           before : after  =      8.7 : 1.0
                   mirna = True           before : after  =      6.1 : 1.0
                  import = True           before : after  =      5.5 : 1.0
                   invas = True           before : after  =      4.8 : 1.0
             overexpress = True           before : after  =      4.8 : 1.0
                  differ = True           before : after  =      4.8 : 1.0
                    base = True            after : before =      3.8 : 1.0
               treatment = True            after : before =      3.8 : 1.0
                 inhibit = True           before : after  =      3.7 : 1.0
                   tissu = True           before : after  =      3.5 : 1.0
               downregul = True           before : after  =      3.5 : 1.0
                  normal = True          

In [116]:
train_Naive_Bayes(documents_50_50_3sentences, list_all_bigrams_50_50_3sentences, 0.20)

documents in test_set: 	 13
documents in training_set: 	 51

Classifier accuracy percent: 46.15384615384615
Most Informative Features
('complex', 'minicellulosom') = False           after : before =      1.0 : 1.0
       ('the', 'reason') = False           after : before =      1.0 : 1.0
 ('whether', 'directli') = False           after : before =      1.0 : 1.0
        ('rate', 'fold') = False           after : before =      1.0 : 1.0
  ('relev', 'pulmonari') = False           after : before =      1.0 : 1.0
   ('splice', 'variant') = False           after : before =      1.0 : 1.0
       ('level', 'vari') = False           after : before =      1.0 : 1.0
        ('the', 'first') = False           after : before =      1.0 : 1.0
       ('regen', 'drug') = False           after : before =      1.0 : 1.0
('cluster', 'differenti') = False           after : before =      1.0 : 1.0
('proinvas', 'prometastat') = False           after : before =      1.0 : 1.0
      ('gain', 'extrem') = False

In [117]:
#### 2.3 - Balanced: 50 % before - 50% after. If doi cited before and after same number of documents before are after.

train_Naive_Bayes(documents_50_50_balanced_3sentences, list_all_words_50_50_balanced_3sentences, 0.20)



documents in test_set: 	 12
documents in training_set: 	 50

Classifier accuracy percent: 66.66666666666666
Most Informative Features
                 patient = True            after : before =     12.6 : 1.0
                  remain = True            after : before =     11.9 : 1.0
                 present = True            after : before =     10.4 : 1.0
                   evalu = True            after : before =      9.0 : 1.0
                function = True           before : after  =      9.0 : 1.0
                  assess = True            after : before =      8.3 : 1.0
                conflict = True            after : before =      7.6 : 1.0
                   other = True            after : before =      7.6 : 1.0
                  induct = True            after : before =      7.6 : 1.0
                individu = True            after : before =      7.6 : 1.0
                research = True           before : after  =      6.5 : 1.0
               discoveri = True          

In [118]:

train_Naive_Bayes(documents_50_50_balanced_3sentences, list_all_bigrams_50_50_balanced_3sentences, 0.20)

documents in test_set: 	 12
documents in training_set: 	 50

Classifier accuracy percent: 41.66666666666667
Most Informative Features
    ('receptor', 'gene') = False           after : before =      1.0 : 1.0
  ('sequenc', 'describ') = False           after : before =      1.0 : 1.0
      ('cancer', 'when') = False           after : before =      1.0 : 1.0
       ('the', 'reason') = False           after : before =      1.0 : 1.0
  ('cytokin', 'monocyt') = False           after : before =      1.0 : 1.0
    ('organ', 'nanowir') = False           after : before =      1.0 : 1.0
        ('rate', 'fold') = False           after : before =      1.0 : 1.0
  ('likelihood', 'fals') = False           after : before =      1.0 : 1.0
('nextgener', 'sequenc') = False           after : before =      1.0 : 1.0
        ('the', 'first') = False           after : before =      1.0 : 1.0
       ('regen', 'drug') = False           after : before =      1.0 : 1.0
    ('improv', 'glucos') = False         

In [119]:
### 3. With one sentence citing + following 3 sentences:

#### 3.1 -  All documents

train_Naive_Bayes(documents_all_5sentences, list_all_words_5sentences, 0.20)


documents in test_set: 	 24
documents in training_set: 	 98

Classifier accuracy percent: 58.333333333333336
Most Informative Features
                   phase = True            after : before =      7.8 : 1.0
                  mainli = True            after : before =      7.8 : 1.0
                concentr = True            after : before =      7.8 : 1.0
                 protect = True            after : before =      7.8 : 1.0
               morpholog = True            after : before =      7.8 : 1.0
                   littl = True            after : before =      7.8 : 1.0
                  despit = True            after : before =      7.4 : 1.0
                    grow = True            after : before =      6.0 : 1.0
                   renal = True            after : before =      5.6 : 1.0
                  extend = True            after : before =      5.6 : 1.0
                     day = True            after : before =      5.6 : 1.0
                    focu = True         

In [120]:

train_Naive_Bayes(documents_all_5sentences, list_all_bigrams_5sentences, 0.20)

documents in test_set: 	 24
documents in training_set: 	 98

Classifier accuracy percent: 58.333333333333336
Most Informative Features
('complex', 'minicellulosom') = False          before : after  =      1.0 : 1.0
    ('influenc', 'meal') = False          before : after  =      1.0 : 1.0
        ('estim', 'use') = False          before : after  =      1.0 : 1.0
       ('not', 'involv') = False          before : after  =      1.0 : 1.0
       ('the', 'reason') = False          before : after  =      1.0 : 1.0
         ('dna', 'herv') = False          before : after  =      1.0 : 1.0
  ('ligand', 'previous') = False          before : after  =      1.0 : 1.0
 ('whether', 'directli') = False          before : after  =      1.0 : 1.0
        ('rate', 'fold') = False          before : after  =      1.0 : 1.0
 ('consensu', 'sequenc') = False          before : after  =      1.0 : 1.0
('express', 'fulllength') = False          before : after  =      1.0 : 1.0
('hyperglycemia', 'devast') = Fals

In [121]:
#### 3.2 -  50 % before - 50% after

train_Naive_Bayes(documents_50_50_5sentences, list_all_words_50_50_5sentences, 0.20)
    

documents in test_set: 	 13
documents in training_set: 	 51

Classifier accuracy percent: 61.53846153846154
Most Informative Features
                     mir = True           before : after  =      8.7 : 1.0
               metastasi = True           before : after  =      6.1 : 1.0
                metastat = True           before : after  =      4.8 : 1.0
                    caus = True            after : before =      4.5 : 1.0
              suppressor = True           before : after  =      4.2 : 1.0
                   mirna = True           before : after  =      4.0 : 1.0
                 control = True           before : after  =      4.0 : 1.0
                   after = True            after : before =      3.8 : 1.0
                 decreas = True            after : before =      3.8 : 1.0
                  target = True           before : after  =      3.7 : 1.0
                 unknown = True           before : after  =      3.5 : 1.0
                 previou = True          

In [123]:
train_Naive_Bayes(documents_50_50_5sentences, list_all_bigrams_50_50_5sentences, 0.20)
    

documents in test_set: 	 13
documents in training_set: 	 51

Classifier accuracy percent: 46.15384615384615
Most Informative Features
('complex', 'minicellulosom') = False          before : after  =      1.0 : 1.0
    ('influenc', 'meal') = False          before : after  =      1.0 : 1.0
       ('the', 'reason') = False          before : after  =      1.0 : 1.0
 ('whether', 'directli') = False          before : after  =      1.0 : 1.0
        ('rate', 'fold') = False          before : after  =      1.0 : 1.0
('express', 'fulllength') = False          before : after  =      1.0 : 1.0
('hyperglycemia', 'devast') = False          before : after  =      1.0 : 1.0
    ('inhibit', 'death') = False          before : after  =      1.0 : 1.0
  ('relev', 'pulmonari') = False          before : after  =      1.0 : 1.0
   ('splice', 'variant') = False          before : after  =      1.0 : 1.0
       ('level', 'vari') = False          before : after  =      1.0 : 1.0
        ('the', 'first') = False

In [128]:
#### 3.3 - Balanced: 50 % before - 50% after. If doi cited before and after same number of documents before are after.

train_Naive_Bayes(documents_50_50_balanced_5sentences, list_all_words_50_50_balanced_5sentences, 0.20)



documents in test_set: 	 12
documents in training_set: 	 50

Classifier accuracy percent: 58.333333333333336
Most Informative Features
                     via = True            after : before =      9.7 : 1.0
                    sinc = True            after : before =      9.7 : 1.0
            characterist = True            after : before =      8.8 : 1.0
                 endogen = True            after : before =      8.8 : 1.0
                  assess = True            after : before =      8.8 : 1.0
                     due = True            after : before =      8.8 : 1.0
                  analyz = True            after : before =      8.0 : 1.0
                conflict = True            after : before =      8.0 : 1.0
            particularli = True            after : before =      8.0 : 1.0
                     msc = True            after : before =      8.0 : 1.0
                   other = True            after : before =      8.0 : 1.0
                    fail = True         

In [126]:
train_Naive_Bayes(documents_50_50_balanced_5sentences, list_all_bigrams_50_50_balanced_5sentences, 0.20)


documents in test_set: 	 12
documents in training_set: 	 50

Classifier accuracy percent: 25.0
Most Informative Features
    ('receptor', 'gene') = False          before : after  =      1.0 : 1.0
    ('influenc', 'meal') = False          before : after  =      1.0 : 1.0
  ('sequenc', 'describ') = False          before : after  =      1.0 : 1.0
      ('cancer', 'when') = False          before : after  =      1.0 : 1.0
       ('the', 'reason') = False          before : after  =      1.0 : 1.0
  ('cytokin', 'monocyt') = False          before : after  =      1.0 : 1.0
    ('organ', 'nanowir') = False          before : after  =      1.0 : 1.0
        ('rate', 'fold') = False          before : after  =      1.0 : 1.0
('hyperglycemia', 'devast') = False          before : after  =      1.0 : 1.0
  ('likelihood', 'fals') = False          before : after  =      1.0 : 1.0
('nextgener', 'sequenc') = False          before : after  =      1.0 : 1.0
        ('raxb', 'serv') = False          before : 